In [1]:
import pandas as pd

### Load comparison data

In [2]:
data = pd.read_excel('C:/Users/glappe/Documents/validated_phones_and_polos_2021-03-23.xlsx')

In [3]:
data.head()

,ME_NO,FNAME,MNAME,LNAME,OFFICE_PHONE,ADDR1,ADDR2,ADDR_CITY,ADDR_STATE,ADDR_ZIP
0,4204880064,CARMEN,J,SIERRA LUCIANO,7872818522,NaN,400 CALLE MANUEL DOMENECH STE 311,SAN JUAN,PR,918
1,3545120088,ERIN,ELIZABETH,CASEY,6364417280,NaN,11 GARVEY PKWY,SAINT CHARLES,MO,63303
2,69007840063,OBIOHA,LOUIS,OKORO,4787466662,NaN,844 2ND ST,MACON,GA,31201
3,65501960025,MOUNIR,NaN,BELCADI,8056526729,NaN,200 HILLMONT AVE,VENTURA,CA,93003
4,802951248,GORDON,SCOTT,ABRAMS,4126411635,MAGEE-WOMENS HOSPITAL,300 HALKET ST,PITTSBURGH,PA,15213


### Get IQVIA data

In [5]:
iqvia_file = 'C:/Users/glappe/Documents/iqvia_org_info.csv'
iqvia = pd.read_csv(iqvia_file, dtype=str)

In [6]:
for c in iqvia.columns:
    iqvia[c] = iqvia[c].astype(str).apply(lambda x: x.upper().strip())
iqvia

,IMS_ORG_ID,BUSINESS_NAME,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHYSICAL_ZIP,PHONE,PROFESSIONAL_ID,LAST_NAME,ME
0,INS00646252,"WATCHI, RALPH, OFFICE",1300 N VERMONT AVE,STE 901,LOS ANGELES,CA,90027-6094,3239129221,830451,WATCHI,1650183012
1,INS00636943,KHASAK DERMATOLOGY,4915 BROADWAY,FRNT 1,NEW YORK,NY,10034-3119,2125678184,923401,KHASAK,0351992056
2,INS00636951,WINSTON SALEM DERMATOLOGY,1400 WESTGATE CENTER DR,STE 200,WINSTON SALEM,NC,27103-3104,3367748636,259973,YUENGEL,0330684108
3,INS00636994,"CATHERINE BOMBERGER, MD, LLC",310 HOSPITAL DR,BLDG B,MACON,GA,31217-3895,4784645000,4629926,DE GUZMAN,7482103001
4,INS00636994,"CATHERINE BOMBERGER, MD, LLC",310 HOSPITAL DR,BLDG B,MACON,GA,31217-3895,4784645000,425893,BOMBERGER,0010693011
...,...,...,...,...,...,...,...,...,...,...,...
2756044,INS00593716,HORIZON EYE CARE PINEVILLE,10520 PARK RD,NAN,CHARLOTTE,NC,28210-8487,7045416127,5393128,RYDER,0411511143
2756045,INS00594183,SERRANO OBGYN,20726 STONE OAK PKWY,UNIT 101,SAN ANTONIO,TX,78258-7552,2105457700,335209,SERRANO,0481390159
2756046,INS00594265,DAVITA MEDICAL GROUP NORTHEAST,8950 DR MARTIN LUTHER KING JR ST N,STE 180,SAINT PETERSBURG,FL,33702-3001,7275768900,3940767,BRENNAN-PRESCOD,4220106364
2756047,INS00594265,DAVITA MEDICAL GROUP NORTHEAST,8950 DR MARTIN LUTHER KING JR ST N,STE 180,SAINT PETERSBURG,FL,33702-3001,7275768900,6828438,OO,2090198026


In [35]:
iqvia['LAST_NAME'].values

array(['WATCHI', 'KHASAK', 'YUENGEL', ..., 'BRENNAN-PRESCOD', 'OO',
       'UNNISSA'], dtype=object)

In [7]:
iqvia['ME10'] = iqvia['ME']
del iqvia['ME']

In [9]:
data['ME_NO'] = data['ME_NO'].apply(lambda x: ('000'+str(x))[-11:])

In [10]:
data['ME10'] = data['ME_NO'].apply(lambda x: x[:10])

In [29]:
data['OFFICE_PHONE'] = data['OFFICE_PHONE'].astype(str)

In [38]:
merge = data[
    ['ME_NO', 'ME10', 'LNAME', 'OFFICE_PHONE']
].merge(
    iqvia[
        ['ME10', 'LAST_NAME', 'PHONE']], 
    left_on=['ME10', 'LNAME'],
    right_on=['ME10', 'LAST_NAME'],
    how='left'
)[['ME_NO', 'OFFICE_PHONE', 'PHONE']]
merge

,ME_NO,OFFICE_PHONE,PHONE
0,04204880064,7872818522,7877588383
1,03545120088,6364417280,6369475000
2,03545120088,6364417280,6364417280
3,03545120088,6364417280,3148370405
4,69007840063,4787466662,4786331000
...,...,...,...
1530,00801930336,3179639300,3176882000
1531,01803940262,5158759192,NaN
1532,04203920119,9372810900,9375344600
1533,04203920119,9372810900,9372263200


### Coverage

In [39]:
iqvia_has_phone = iqvia[
    ~iqvia['PHONE'].isna()
]['ME10'].drop_duplicates()
len(iqvia_has_phone)

1000832

In [41]:
coverage = data['ME10'].isin(iqvia_has_phone).sum()
coverage

483

In [42]:
coverage / len(data)

0.966

### Match Rate

In [43]:
me_phones_mf = data[['ME_NO', 'OFFICE_PHONE']].drop_duplicates()
me_phones_iqvia = merge[['ME_NO', 'PHONE']].drop_duplicates()

In [45]:
matches = me_phones_mf.set_index(['ME_NO', 'OFFICE_PHONE']).index.isin(
    me_phones_iqvia.set_index(['ME_NO', 'PHONE']).index
).sum()

matches

297

In [46]:
matches / len(data)

0.594

In [48]:
merge.rename(columns={'PHONE': 'IQVIA_PHONE'}).to_csv('iqvia_comparison.csv', index=False)